In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
engaged_df=pd.read_csv('../Data/Student-engagement-dataset/engaged.csv')
notEngaged_df = pd.read_csv('../Data/Student-engagement-dataset/notEngaged.csv')

In [3]:
engaged_x=engaged_df.drop(['engaged','path','state'],axis=1)
engaged_y=engaged_df['engaged']

notEngaged_x = notEngaged_df.drop(['engaged', 'path', 'state'], axis=1)
notEngaged_y = notEngaged_df['engaged']

In [4]:
engaged_X_train, engaged_X_test, engaged_y_train, engaged_y_test = train_test_split(
    engaged_x, engaged_y, train_size=0.8, random_state=42)

notEngaged_X_train, notEngaged_X_test, notEngaged_y_train, notEngaged_y_test = train_test_split(
    notEngaged_x, notEngaged_y, train_size=0.8, random_state=42)

In [5]:
X_train=pd.concat([engaged_X_train,notEngaged_X_train],axis=0)
X_test = pd.concat([engaged_X_test, notEngaged_X_test], axis=0)
y_train = pd.concat([engaged_y_train, notEngaged_y_train], axis=0)
y_test = pd.concat([engaged_y_test, notEngaged_y_test], axis=0)

In [58]:
X_train.drop(['mouthSmileLeft',	'mouthSmileRight',	'mouthStretchLeft',
             'mouthStretchRight',	'mouthUpperUpLeft',	'mouthUpperUpRight']).head(5)

,browDownLeft,browDownRight,browInnerUp,browOuterUpLeft,browOuterUpRight,eyeBlinkLeft,eyeBlinkRight,eyeLookDownLeft,eyeLookDownRight,eyeLookInLeft,...,eyeSquintRight,eyeWideLeft,eyeWideRight,jawOpen,mouthSmileLeft,mouthSmileRight,mouthStretchLeft,mouthStretchRight,mouthUpperUpLeft,mouthUpperUpRight
109,0.052787,0.039122,0.100689,0.055260,0.054874,0.167119,0.116762,0.446202,0.419401,0.494345,...,0.207851,0.004410,0.006238,0.000252,0.000838,0.000859,0.002854,0.002791,0.003863,0.006152
77,0.039426,0.045134,0.094147,0.043793,0.030591,0.357934,0.243603,0.576784,0.568430,0.019664,...,0.364018,0.002015,0.001962,0.000138,0.005712,0.005367,0.009416,0.003408,0.006008,0.012528
549,0.000210,0.002918,0.249254,0.717393,0.218742,0.103115,0.102125,0.333779,0.342668,0.018674,...,0.135368,0.011824,0.006030,0.000165,0.632421,0.695958,0.009428,0.028890,0.021330,0.034349
843,0.000080,0.016708,0.139541,0.741690,0.069874,0.166616,0.271982,0.583993,0.575388,0.043783,...,0.254356,0.007658,0.001982,0.000239,0.762088,0.796860,0.010564,0.012066,0.027106,0.050674
82,0.059523,0.068909,0.069666,0.031470,0.021055,0.338086,0.256635,0.505538,0.494552,0.035207,...,0.347204,0.002430,0.001765,0.000210,0.001061,0.000970,0.003027,0.001275,0.003145,0.006285


In [11]:
## just testing

from sklearn.ensemble import RandomForestClassifier

rnd_forest=RandomForestClassifier(n_estimators=10)

rnd_forest.fit(X_train,y_train)

RandomForestClassifier(n_estimators=10)

In [12]:
rnd_forest.feature_importances_

array([0.03775773, 0.01610235, 0.02249245, 0.02207373, 0.02690496,
       0.02692287, 0.0353981 , 0.02555327, 0.04213188, 0.01403218,
       0.01020585, 0.02352508, 0.01625645, 0.01791895, 0.01807598,
       0.03443893, 0.0103091 , 0.03605661, 0.05241493, 0.02451279,
       0.13573123, 0.03824286, 0.03526249, 0.02075045, 0.18173043,
       0.07519837])

In [17]:
rnd_forest.score(X_test,y_test)

0.965625

In [17]:
from sklearn.svm import SVC

svc=SVC(probability=True,kernel='rbf')

svc.fit(X_train,y_train)

SVC(probability=True)

In [19]:
svc.score(X_test,y_test)

0.953125

In [32]:
import sys
sys.path.insert(1, '../')

from commonfunctions import *

In [33]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import skimage.io as IO
import cv2
import os
import pandas as pd

In [34]:
base_options = python.BaseOptions(model_asset_path='../MediaPipe/face_landmarker.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)


def detect(path):
    image = mp.Image.create_from_file(path)

    detection_result = detector.detect(image)

    annotated_image = draw_landmarks_on_image(
        image.numpy_view(), detection_result)
    return annotated_image, detection_result

In [35]:
path = "../Data/Student-engagement-dataset/Engaged/confused/0021.jpg"
image, result = detect(path)

In [36]:
face_blendshapes_names = [
    face_blendshapes_category.category_name for face_blendshapes_category in result.face_blendshapes[0]]
face_blendshapes_scores = [
    face_blendshapes_category.score for face_blendshapes_category in result.face_blendshapes[0]]
face_blendshapes_ranks = range(len(face_blendshapes_names))

In [37]:

needed_names = [face_blendshapes_names[i] for i in [1, 2, 3, 4, 5, 9, 10, 11,
                                                    12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 44, 45, 46, 47, 48, 49]]

needed_scores = [face_blendshapes_scores[i] for i in [1, 2, 3, 4, 5, 9, 10,
                                                      11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 44, 45, 46, 47, 48, 49]]
# for name, score in zip(needed_names, needed_scores):
#     print(f"{name}: {score}")

In [31]:
np.array(X_train.head(1))

array([[5.27868308e-02, 3.91216949e-02, 1.00689486e-01, 5.52598089e-02,
        5.48736639e-02, 1.67119369e-01, 1.16762161e-01, 4.46201712e-01,
        4.19400632e-01, 4.94345099e-01, 3.59314680e-03, 7.64214806e-03,
        4.74294603e-01, 1.49952695e-02, 1.34808309e-02, 2.48957589e-01,
        2.07850814e-01, 4.41030599e-03, 6.23841444e-03, 2.52401514e-04,
        8.38337932e-04, 8.59036460e-04, 2.85428599e-03, 2.79062684e-03,
        3.86311230e-03, 6.15163660e-03]])

In [32]:
rnd_forest.predict([needed_scores])

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([1], dtype=int64)

In [8]:
import pickle

# pickle.dump(rnd_forest, open("random_forest.pkl", "wb"))

pickle.dump(svc, open("svc.pkl", "wb"))



In [34]:
model = pickle.load(open(
    "D:\GitHub Repos\Face-Emotion-Vision\studentEngagementModel\\random_forest.pkl", "rb"))

In [35]:
model.predict([needed_scores])

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([1], dtype=int64)

In [37]:
path = "../Data/Student-engagement-dataset/Engaged/confused/0021.jpg"
image, result = detect(path)

face_blendshapes_names = [
    face_blendshapes_category.category_name for face_blendshapes_category in result.face_blendshapes[0]]
face_blendshapes_scores = [
    face_blendshapes_category.score for face_blendshapes_category in result.face_blendshapes[0]]


needed_names = [face_blendshapes_names[i] for i in [1, 2, 3, 4, 5, 9, 10, 11,
                                                    12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 44, 45, 46, 47, 48, 49]]

needed_scores = [face_blendshapes_scores[i] for i in [1, 2, 3, 4, 5, 9, 10,
                                                      11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 44, 45, 46, 47, 48, 49]]

if model.predict([needed_scores]) == 1:
    print("Engaged")
else:
    print("Not Engaged")

Engaged


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [68]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


voting_clf = VotingClassifier(
    estimators=[
        # ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42))
    ]
)
# voting_clf.fit(X_train.values, y_train)

voting_clf.fit(X_train.drop(['mouthSmileLeft',	'mouthSmileRight',	'mouthStretchLeft',
             'mouthStretchRight',	'mouthUpperUpLeft',	'mouthUpperUpRight'],axis=1).values, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [64]:
voting_clf.score(X_test.drop(['mouthSmileLeft',	'mouthSmileRight',	'mouthStretchLeft',
             'mouthStretchRight',	'mouthUpperUpLeft',	'mouthUpperUpRight'], axis=1).values,y_test)

0.93125

In [48]:
for name, clf in voting_clf.named_estimators_.items():
    print(name, "=", clf.score(X_test, y_test))

lr = 0.890625
rf = 0.965625
svc = 0.953125


c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [69]:
voting_clf.voting = "soft"
voting_clf.named_estimators["svc"].probability = True
voting_clf.fit(X_train.drop(['mouthSmileLeft',	'mouthSmileRight',	'mouthStretchLeft',
                             'mouthStretchRight',	'mouthUpperUpLeft',	'mouthUpperUpRight'], axis=1).values, y_train)
voting_clf.score(X_test.drop(['mouthSmileLeft',	'mouthSmileRight',	'mouthStretchLeft',
                             'mouthStretchRight',	'mouthUpperUpLeft',	'mouthUpperUpRight'], axis=1).values, y_test)

0.953125

In [25]:
for score, name in zip(rnd_forest.feature_importances_, rnd_forest.feature_names_in_):
    print(round(score, 2), name)

0.04 browDownLeft
0.02 browDownRight
0.02 browInnerUp
0.02 browOuterUpLeft
0.03 browOuterUpRight
0.03 eyeBlinkLeft
0.04 eyeBlinkRight
0.03 eyeLookDownLeft
0.04 eyeLookDownRight
0.01 eyeLookInLeft
0.01 eyeLookInRight
0.02 eyeLookOutLeft
0.02 eyeLookOutRight
0.02 eyeLookUpLeft
0.02 eyeLookUpRight
0.03 eyeSquintLeft
0.01 eyeSquintRight
0.04 eyeWideLeft
0.05 eyeWideRight
0.02 jawOpen
0.14 mouthSmileLeft
0.04 mouthSmileRight
0.04 mouthStretchLeft
0.02 mouthStretchRight
0.18 mouthUpperUpLeft
0.08 mouthUpperUpRight


In [70]:
voting_clf.predict_proba([needed_scores])

ValueError: X has 26 features, but RandomForestClassifier is expecting 20 features as input.

In [2]:
import pickle


pickle.dump(voting_clf, open("voting_clf.pkl", "wb"))

In [3]:
models = {
    1: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\KNN_final.pkl", "rb")),
    2: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\KNN_hog.pkl", "rb")),
    3: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\KNN_lbp.pkl", "rb")),
    4: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\\rnd_final.pkl", "rb")),
    5: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\\rnd_hog.pkl", "rb")),
    6: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\\rnd_lbp.pkl", "rb")),
    7: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\SVM_final.pkl", "rb")),
    8: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\SVM_hog.pkl", "rb")),
    9: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\SVM_lbp.pkl", "rb")),
    10: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\\voting_clf_hard_final.pkl", "rb")),
    11: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\\voting_clf_hard_hog.pkl", "rb")),
    12: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\\voting_clf_hard_lbp.pkl", "rb")),
    13: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\\voting_clf_soft_final.pkl", "rb")),
    14: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\\voting_clf_soft_hog.pkl", "rb")),
    15: pickle.load(open(
        "D:\GitHub Repos\Face-Emotion-Vision\playingWithFaceRecognition\\voting_clf_soft_lbp.pkl", "rb")),
}

In [6]:
models[1]

<function dict.values>